# **In this lab, you will understand the basics of <font color="red"> Langchain. </font>**

Install the below libraries required for this lab.

In [ ]:
pip install langchain langchain_openai langsmith python-dotenv

If you are using Jupyter notebook, follow the below instructions. Else skip this step and go to next step

**Open .env file in this folder and observe that we have configured OPENAI_API_KEY. Replace it with your own key or key given by me**

The Code in the below cell will load the .env file and set environment variables.

**Write the code in the below cell and execute it**

In [1]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

# Access the 'OPENAI_API_KEY' from the environment
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
 print("OPENAI_API_KEY  in environment variables is ",openai_api_key)
else:
    print("OPENAI_API_KEY not found in environment variables.")


OPENAI_API_KEY  in environment variables is  6i1aCDa8k4eT7RvAz7XvSpJB8EQo9gKK7EoH5G5f92V8onnC7I5nJQQJ99BDACYeBjFXJ3w3AAAAACOGkbOc


Ignore the below if you are not using Azure OpenAI

In [2]:
azure_openai_api_key = "33lOhJMyWY9Q2mylbg8OCRsc9jsmlF5r5XBC5eVVDJifssj5XXgPJQQJ99BDACfhMk5XJ3w3AAAAACOGULII"
azure_openai_endpoint = "https://sivap-m92qj7xv-swedencentral.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview"
azure_openai_deployment = "gpt-4o-mini" # Your GPT-4o-mini deployment name


# We want to send messages using GPT API

Write code in the below cell, understand it and execute it.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content="Translate the following from English into Hindi"),
    HumanMessage(content="Hello!"),
]
llm = ChatOpenAI()

result = llm.invoke(messages)
result

**Notice that the response from the model is an AIMessage.**

This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response.
Understand and execute below code

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = llm.invoke(messages)
parser.invoke(result)

**We can create a chain such that the output of invoking llm will be given as input to parser.**

Understand and execute below code

In [ ]:
chain = llm | parser
chain.invoke(messages)

# Understanding Prompt Templates

Let's create a PromptTemplate here. It will take in two user variables:

<font color="red"> **language** </font>: The language to translate text into

<font color="red"> **text** </font>: The text to translate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate



prompt_template = ChatPromptTemplate(
    [("system", "Translate the following into {language}:"),
    ("user", "{text}")]
)

result = prompt_template.invoke({"language": "chinese", "text": "hi"})

result

## Chaining prompt template, llm and parser

In [ ]:
chain = prompt_template | llm | parser
chain.invoke({"language": "chinese", "text": "hi"})

See this link to understand how to use **FewShotPromptTemplate.**

https://python.langchain.com/docs/how_to/few_shot_examples/

## Define Data Models and using pydantic parser for generating format instructions
Define the data models using <font color="red"> **Pydantic** </font> to structure the information about people as shown below

Use the below code and execute it in new cell

In [ ]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(description="The name of the person")
    height_in_meters: float = Field(
        description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

print(parser.get_format_instructions())


In [ ]:

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

from IPython.display import HTML,Markdown

Markdown(prompt.invoke({"query": "Rishik is 13 years old and she is 6 feet tall . Siva is 43 years old and 5.7 feet tall"}).messages[0].content)

# Using the above prompt , create a chain  and invoke it to get json response

Use the below code and execute it in new cell

In [ ]:
chain = prompt | llm
query = "Rishik is 13 years old and hhe is 6 feet tall . Siva is 43 years old and 5.6 feet tall"

chain.invoke({"query": query})

from IPython.display import HTML,Markdown

result=chain.invoke({"query": query})
(Markdown(result.content))


***We can cache the llm responses. See the below URL to understand about caching***

https://python.langchain.com/docs/how_to/chat_model_caching/

In [ ]:
pip install -U langchain-community

In [ ]:
%%time

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())


In [ ]:
%%time
llm.invoke("Tell me a joke")

In [ ]:
%%time
llm.invoke("Tell me a joke")